In [1]:
import pandas as pd

In [20]:
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [21]:
data.drop(columns=["RowNumber","CustomerId","Surname"],inplace=True)

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [4]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,StandardScaler

labelEncoder=LabelEncoder()
gender=labelEncoder.fit_transform(data[["Gender"]])

c:\Users\spasula\AppData\Local\miniconda3\envs\venv\lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [5]:
data['Gender']=gender
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [6]:
data['Gender'].value_counts()

Gender
1    5457
0    4543
Name: count, dtype: int64

In [7]:
onehotencoder=OneHotEncoder()
Geography=onehotencoder.fit_transform(data[['Geography']])
Geography.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [8]:
data=pd.concat([data.drop(columns=['Geography'],axis=1),pd.DataFrame(Geography.toarray(),columns=onehotencoder.get_feature_names_out(['Geography']))],axis=1)
data.columns

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited',
       'Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype='object')

In [9]:
from sklearn.model_selection import train_test_split

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
standardscaler=StandardScaler()
X_train=pd.DataFrame(standardscaler.fit_transform(X_train),columns=X_train.columns)


In [12]:
X_test=pd.DataFrame(standardscaler.transform(X_test),columns=X_test.columns)

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [26]:
from sklearn.metrics import accuracy_score,classification_report

In [27]:
lg=LogisticRegression()
lg.fit(X_train,y_train)
y_pred=lg.predict(X_test)
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.811
              precision    recall  f1-score   support

           0       0.83      0.96      0.89      1607
           1       0.55      0.20      0.29       393

    accuracy                           0.81      2000
   macro avg       0.69      0.58      0.59      2000
weighted avg       0.78      0.81      0.77      2000



In [29]:
rfc=RandomForestClassifier()
rfc.fit(X_train,y_train)
y_pred=rfc.predict(X_test)
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.868
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      1607
           1       0.76      0.48      0.59       393

    accuracy                           0.87      2000
   macro avg       0.82      0.72      0.76      2000
weighted avg       0.86      0.87      0.86      2000



In [30]:
import xgboost as xgb
from xgboost import XGBClassifier
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.86


c:\Users\spasula\AppData\Local\miniconda3\envs\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [17:54:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [49]:
import pickle

In [50]:
with open('./picklefiles/lableencoder.pkl','wb') as file:
    pickle.dump(labelEncoder,file)

In [51]:
with open('./picklefiles/onehotencoder.pkl','wb') as file:
    pickle.dump(onehotencoder,file)

In [52]:
with open('./picklefiles/standardscaler.pkl','wb') as file:
    pickle.dump(standardscaler,file)

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [17]:
from tensorflow.keras.callbacks import EarlyStopping

In [31]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

In [14]:
model=Sequential(
    [
        Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
        Dense(32,activation='relu'),
        Dense(1,activation='sigmoid')

    ]
)

c:\Users\spasula\AppData\Local\miniconda3\envs\venv\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

In [19]:
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8823 - loss: 0.2798 - val_accuracy: 0.8565 - val_loss: 0.3424
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8823 - loss: 0.2873 - val_accuracy: 0.8575 - val_loss: 0.3453
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8748 - loss: 0.2889 - val_accuracy: 0.8620 - val_loss: 0.3389
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8749 - loss: 0.2996 - val_accuracy: 0.8640 - val_loss: 0.3444
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8770 - loss: 0.2925 - val_accuracy: 0.8625 - val_loss: 0.3451
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8774 - loss: 0.2855 - val_accuracy: 0.8620 - val_loss: 0.3440
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8831 - loss: 0.2803 - val_accuracy: 0.8640 - val_loss: 0.3420
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8831 - loss: 0.2824 - val_accu